## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import cv2
import os

## Preparar os dados

In [ ]:
# Configurações gerais
R = 50  # Redimensionamento das imagens
base_path = "data\RecFac"

(1400, 3)


In [ ]:
name_list = os.listdir(base_path)
C = len(name_list)  # Número de classes
X = np.empty((R*R, 0))  # Dados p x N
Y = np.empty((C, 0))    # Rótulos one-hot C x N

for i, nome in enumerate(name_list):
    lista_imagens = os.listdir(f"{base_path}\\{nome}")
    
    # Criação do vetor one-hot
    rotulo = -np.ones((C, len(lista_imagens)))
    rotulo[i, :] = 1
    Y = np.hstack((Y, rotulo))
    
    # Processamento das imagens
    for imagem in lista_imagens:
        img = cv2.imread(f"{base_path}\\{nome}\\{imagem}", cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (R, R))
        x = img.flatten().reshape(img.size, 1)
        X = np.hstack((X, x))

# Normalização dos dados entre 0 e 1
X = (X - np.min(X)) / (np.max(X) - np.min(X))
print(f"X: {X.shape}, Y: {Y.shape}")name_list = os.listdir(base_path)
name_list = os.listdir(base_path)
C = len(name_list)  # Número de classes
X = np.empty((R*R, 0))  # Dados p x N
Y = np.empty((C, 0))    # Rótulos one-hot C x N

for i, nome in enumerate(name_list):
    lista_imagens = os.listdir(f"{base_path}\\{nome}")
    
    # Criação do vetor one-hot
    rotulo = -np.ones((C, len(lista_imagens)))
    rotulo[i, :] = 1
    Y = np.hstack((Y, rotulo))
    
    # Processamento das imagens
    for imagem in lista_imagens:
        img = cv2.imread(f"{base_path}\\{nome}\\{imagem}", cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (R, R))
        x = img.flatten().reshape(img.size, 1)
        X = np.hstack((X, x))

# Normalização dos dados entre 0 e 1
X = (X - np.min(X)) / (np.max(X) - np.min(X))
print(f"X: {X.shape}, Y: {Y.shape}")

## Funções auxiliares

In [ ]:
# Função para calcular acurácia
def accuracy(y_true, y_pred):
    pred_class = np.argmax(y_pred, axis=0)
    true_class = np.argmax(y_true, axis=0)
    return np.mean(pred_class == true_class)

# Função para criar matriz de confusão
def confusion_matrix(y_true, y_pred):
    pred_class = np.argmax(y_pred, axis=0)
    true_class = np.argmax(y_true, axis=0)
    C = y_true.shape[0]
    cm = np.zeros((C, C), dtype=int)
    for t, p in zip(true_class, pred_class):
        cm[t, p] += 1
    return cm

In [ ]:
def monte_carlo_split(X, Y, train_ratio=0.8):
    N = X.shape[1]
    idx = np.random.permutation(N)
    train_size = int(train_ratio * N)
    train_idx = idx[:train_size]
    test_idx = idx[train_size:]
    return X[:, train_idx], Y[:, train_idx], X[:, test_idx], Y[:, test_idx]


## Classes

In [11]:
class Perceptron:
    def __init__(self, X_train, y_train, learning_rate=1e-3, plot=True):
        self.p, self.N = X_train.shape
        self.X_train = np.vstack((
            -np.ones((1, self.N)), X_train
        ))
        self.d = y_train
        self.lr = learning_rate
        self.w = np.random.random_sample((self.p+1,1)) - 0.5
        self.plot = plot
        self.x1 = np.linspace(-2, 10)
        
        if plot:
            self.fig = plt.figure(1)
            self.ax = self.fig.add_subplot()
            self.ax.scatter(self.X_train[1, self.d[:]==1],
                            self.X_train[2, self.d[:]==1], marker='s', s=120)
            self.ax.scatter(self.X_train[1, self.d[:]==-1],
                            self.X_train[2, self.d[:]==-1], marker='o', s=120)
            self.ax.set_xlim(-1, 7)
            self.ax.set_ylim(-1, 7)
            self.draw_line()
        
    def draw_line(self, c='k', alpha=1, lw=2):
        x2 = -self.w[1,0]/self.w[2,0]*self.x1 + self.w[0,0]/self.w[2,0]
        x2 = np.nan_to_num(x2)
        if self.plot:
            plt.plot(self.x1, x2, c=c, alpha=alpha, lw=lw)
        
    def activation_function(self, u):
        return 1 if u >= 0 else -1
    
    def fit(self):
        epochs = 0
        error = True
        while error:
            error = False
            for k in range(self.N):
                x_k = self.X_train[:, k].reshape(self.p+1, 1)
                u_k = (self.w.T @ x_k)[0,0]
                y_k = self.activation_function(u_k)
                d_k = self.d[k]
                e_k = d_k - y_k
                if e_k != 0:
                    error = True
                self.w = self.w + self.lr * e_k * x_k
            
            if self.plot:
                plt.pause(.4)
                self.draw_line(c='r', alpha=.5)
            epochs += 1
        
        if self.plot:
            plt.pause(.4)
            self.draw_line(c='g', alpha=1, lw=4)
            plt.show()
        print(f'Treinamento concluído em {epochs} épocas.')

In [6]:
class MultilayerPerceptron:
    def __init__(self,X_train:np.ndarray, Y_train:np.ndarray, topology:list, learning_rate = 1e-3, max_epoch=10000, tol = 1e-12):
        '''
        X_train (p x N)
        Y_train (C x N) ou (1 x N) se classificação binária
        '''
        self.p , self.N = X_train.shape
        self.m = Y_train.shape[0]
        
        self.X_train = np.vstack((
            -np.ones((1,self.N)),X_train
        ))
        self.tol = tol
        self.lr = learning_rate
        self.d = Y_train
        topology.append(self.m)
        self.W = [None]*len(topology)
        Z = 0
        for i in range(len(self.W)):
            if i == 0:
                W = np.random.random_sample((topology[i],self.p+1))-.5
            else:
                W = np.random.random_sample((topology[i],topology[i-1]+1))-.5
            self.W[i] = W
            Z += W.size
        print(f"Rede MLP com {Z} parâmetros")
        self.max_epoch = max_epoch
        self.y = [None]*len(topology)
        self.u = [None]*len(topology)
        self.delta = [None]*len(topology)
        
    def g(self, u):
        return (1-np.exp(-u))/(1+np.exp(-u))
    
    def g_d(self, u):
        y = self.g(u)
        return .5*(1-y**2)
    
    def backward(self, e,x):
        for i in range(len(self.W)-1,-1,-1):
            if i == len(self.W)-1:
                self.delta[i] = self.g_d(self.u[i]) * e
                yb = np.vstack((
                    -np.ones((1,1)),
                    self.y[i-1]
                ))
                self.W[i] = self.W[i] + self.lr*(self.delta[i]@yb.T)
            elif i == 0:
                Wnbt = (self.W[i+1][:,1:]).T
                self.delta[i] = self.g_d(self.u[i]) * (Wnbt@self.delta[i+1])
                self.W[i] = self.W[i] + self.lr*(self.delta[i]@x.T)
                
            else:
                Wnbt = (self.W[i+1][:,1:]).T
                self.delta[i] = self.g_d(self.u[i]) * (Wnbt@self.delta[i+1])
                yb = np.vstack((
                    -np.ones((1,1)),
                    self.y[i-1]
                ))
                self.W[i] = self.W[i] + self.lr*(self.delta[i]@yb.T)
            
    
    def forward(self, x):
        
        for i,W in enumerate(self.W):
            if i == 0:
                self.u[i] = W@x
            else:
                yb = np.vstack((
                    -np.ones((1,1)), self.y[i-1]
                ))
                self.u[i] = W@yb                
            self.y[i] = self.g(self.u[i])
         
        
        
    def EQM(self):
        s = 0
        for k in range(self.N):
            x_k = self.X_train[:,k].reshape(self.p+1,1)
            self.forward(x_k)
            y = self.y[-1]
            d = self.d[:,k].reshape(self.m,1)
            e = d - y
            s += np.sum(e**2)
        return s/(2*self.N)
        
    def fit(self):
        epoch = 0
        EQM1 = 1
        self.history = []
        
        while epoch < self.max_epoch and EQM1>self.tol:
            t1 = time()
            for k in range(self.N):
                x_k = self.X_train[:,k].reshape(self.p+1,1)
                #Forward
                self.forward(x_k)
                y = self.y[-1]
                d = self.d[:,k].reshape(self.m,1)
                e = d - y
                #Backward
                self.backward(e,x_k)
            t2 = time()
            EQM1 = self.EQM()
            self.history.append(EQM1)
            epoch+=1
            print(f"Tempo: {t2-t1:.5f}s  Época: {epoch}, EQM: {EQM1:.15f}")

    def predict(self, X):
        """
        Faz a predição para novos dados X (p x N_test)
        Retorna uma matriz de saída (m x N_test)
        """
        N_test = X.shape[1]

        X_bias = np.vstack((-np.ones((1, N_test)), X))
        
        Y_pred = np.zeros((self.m, N_test))
        
        for k in range(N_test):
            x_k = X_bias[:, k].reshape(self.p+1, 1)
            self.forward(x_k)
            Y_pred[:, k] = self.y[-1][:, 0]
        
        return Y_pred


## Treinamentos

### Treinamento do Perceptron Simples

In [ ]:
# Lista para armazenar resultados
R_simulations = 10
acc_perceptron = []

for r in range(R_simulations):
    X_train, Y_train, X_test, Y_test = monte_carlo_split(X, Y)
    
    # Para simplificação, usamos a primeira classe (binário) no Perceptron
    # Cada classe pode ser treinada como one-vs-rest
    results_class = []
    for c in range(Y.shape[0]):
        perc = Perceptron(X_train, Y_train[c, :], learning_rate=0.001, plot=False)
        perc.fit()
        # Previsão
        y_pred = np.array([perc.activation_function(perc.w.T @ np.vstack((-1, X_test[:, k].reshape(-1,1)))) 
                           for k in range(X_test.shape[1])])
        # Converter -1/1 para 0/1
        y_pred_bin = np.where(y_pred==1, 1, -1)
        results_class.append(y_pred_bin.reshape(1, -1))
    
    y_pred_all = np.vstack(results_class)
    acc = accuracy(Y_test, y_pred_all)
    acc_perceptron.append(acc)

print("Acurácia média Perceptron:", np.mean(acc_perceptron))

### Treinamento do MLP

In [ ]:
acc_mlp = []
histories = []

for r in range(R_simulations):
    X_train, Y_train, X_test, Y_test = monte_carlo_split(X, Y)
    
    # Topologia: 1 camada oculta com 50 neurônios
    mlp = MultilayerPerceptron(X_train, Y_train, topology=[50], learning_rate=0.001, max_epoch=500)
    mlp.fit()
    
    # Previsão
    Y_pred = mlp.predict(X_test)
    acc = accuracy(Y_test, Y_pred)
    
    acc_mlp.append(acc)
    histories.append(mlp.history)

print("Acurácia média MLP:", np.mean(acc_mlp))


## Resultados

In [ ]:
# Exemplo para o MLP
idx_max = np.argmax(acc_mlp)
idx_min = np.argmin(acc_mlp)

_, Y_train_max, X_test_max, Y_test_max = monte_carlo_split(X, Y)
Y_pred_max = mlp.predict(X_test_max)
cm_max = confusion_matrix(Y_test_max, Y_pred_max)

_, Y_train_min, X_test_min, Y_test_min = monte_carlo_split(X, Y)
Y_pred_min = mlp.predict(X_test_min)
cm_min = confusion_matrix(Y_test_min, Y_pred_min)

# Plot com seaborn
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.heatmap(cm_max, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz Confusão - Maior Acurácia')

plt.subplot(1,2,2)
sns.heatmap(cm_min, annot=True, fmt='d', cmap='Reds')
plt.title('Matriz Confusão - Menor Acurácia')
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=[acc_perceptron, acc_mlp])
plt.xticks([0,1], ['Perceptron', 'MLP'])
plt.ylabel("Acurácia")
plt.title("Comparação de Acurácia (R=10 rodadas)")
plt.show()
